In [1]:
import FlowRbf as flow
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def f(x):
    return 8 * np.pi**2 * np.sin(2 * np.pi * x[0]) * np.sin(2 * np.pi * x[1])


class Poisson(flow.regions.Region):
    def operator(self, rbf, x, y):
        return - rbf.laplace(x, y), f(x)


class PoissonBoundary(flow.regions.Region):
    def operator(self, rbf, x, y):
        return rbf.func(x, y), 0.0


In [3]:
!export XDG_SESSION_TYPE=x11

# Set the number of grid points along each axis
nx = 20  # number of points along x
ny = 20  # number of points along y

# Generate grid points in [0,1] x [0,1]
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
X, Y = np.meshgrid(x, y)

# Flatten the grid arrays and create a (N, 3) array of 3D points (with z=0)
points = np.vstack((X.ravel(), Y.ravel(), np.zeros(X.size))).T

# Create a tolerance for comparing floating point numbers
tol = 1e-8

# Identify boundary points:
# A point is on the boundary if its x-coordinate is 0 or 1, or its y-coordinate is 0 or 1.
boundary_mask = (
    (np.abs(points[:, 0] - 0) < tol) |
    (np.abs(points[:, 0] - 1) < tol) |
    (np.abs(points[:, 1] - 0) < tol) |
    (np.abs(points[:, 1] - 1) < tol)
)

# Separate boundary and interior points
boundary_points = points[boundary_mask]
interior_points = points[~boundary_mask]

: 

In [ ]:
centres = flow.Centres()
centres.add(PoissonBoundary(), boundary_points)
centres.add(Poisson(), interior_points)